# EJERCICIO CNNS

Teneis que hacer un modelo de ___convulsiones___. El dataset lo teneis aqui:

https://www.kaggle.com/c/cifar-10/data

IMPORTANTE: No useis el test.7z. Descargaos solo el train.7z y las train_labels. Despues ya dividis en train-test y con ese test ya sacais las metricas finales.

QUE OS PIDO:
- Descargad el dataset
- Leedlo en el NB (podeis investigar como hacerlo o usad el NB de landscapes como punto de partida)
- Haced train-test y escalar si procede
- Montad un modelo que tenga: Al menos 3 bloques convolucionales (Conv2D+Pooling+Dropout). PERO TIENE QUE SER UN MODELO FUNCIONAL (https://keras.io/guides/functional_api/). Es una chorrada, pero teneis que investigar la sintaxis nueva.
- Clasificacion, 10 clases, 0.1 de validation split... ya sabeis.
- Entrenad con los parametros que querais e id jugando con el numero de epocas, dropout, o incluso mas o menos capas, numero de filtros... etc... (lo hemos visto todo en clase)
- Evaluad el modelo con las metricas que creais convenientes


NOTAS: 
- Las imagenes son de 32x32
- Son a color luego el shape ya sabeis...
- Recordad que la primera capa debe llevar el input_shape
- Si haceis una red muy grande puede ir lento, asi que empezad despacito y creced si procede
- Usad la historia de epocas (train_loss vs val_loss) para ver si estais overfiteando
- No es imprescindible pero muy muy muy recomendable que hagais el ejercicio en un google colab. Tendreis que ver como subir los datos etc.. pero las epocas iran muuucho mas rapidas y asi aprendeis. Podeis hacerlo local, ver que funciona y migrar a colab, o directamente hacerlo todo en colab
- En teoria, salvo que hay que declarar el modelo Functional, todo lo hemos visto ya. Intentad probar cosas e intentad no abusar de Chatgpt. Daos la oportunidad de hacerlo vosotros y demostraros que podeis. 

- Ah! El ejercicio es OBLIGATORIO. Teneis 3 dias y una tarde para hacerlo, no hay excusas. Todos los aprendizajes de hacer este NB los voy a dar por sabidos. No lo corregiré en clase, pero resolveré todas las dudas que traigais, eso si.

Si durante el finde os surgen dudas me escribis. Pero me gustaria que intentarais resolverlo con la maxima autonomia. Pero escribidme si os atascais.

### Description

CIFAR-10  is an established computer-vision dataset used for object recognition. It is a subset of the 80 million tiny images dataset and consists of 60,000 32x32 color images containing one of 10 object classes, with 6000 images per class. It was collected by Alex Krizhevsky, Vinod Nair, and Geoffrey Hinton.


The CIFAR-10 data consists of 60,000 32x32 color images in 10 classes, with 6000 images per class. There are 50,000 training images and 10,000 test images in the official data. We have preserved the train/test split from the original dataset. 

 The provided files are:

- train.7z - a folder containing the training images in png format
- trainLabels.csv - the training labels

The label classes in the dataset are:

- airplane 
- automobile 
- bird 
- cat 
- deer 
- dog 
- frog 
- horse 
- ship 
- truck

The classes are completely mutually exclusive. There is no overlap between automobiles and trucks. "Automobile" includes sedans, SUVs, things of that sort. "Truck" includes only big trucks. Neither includes pickup trucks.

In [26]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Cargar el dataset CIFAR-10
from tensorflow.keras.datasets import cifar10

(x_train, y_train), (x_test, y_test) = cifar10.load_data()


In [29]:
y_train

array([[6],
       [9],
       [9],
       ...,
       [9],
       [1],
       [1]], dtype=uint8)

In [30]:

# Escalar los datos
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0

y_train = y_train.reshape(-1)
y_test = y_test.reshape(-1)


In [31]:
y_train

array([6, 9, 9, ..., 9, 1, 1], dtype=uint8)

In [33]:

# Modelo funcional
input_layer = Input(shape=(32, 32, 3))

# Bloque 1
x = Conv2D(32, (3, 3), activation='relu', padding='same')(input_layer)
x = MaxPooling2D((2, 2))(x)
x = Dropout(0.25)(x)

# Bloque 2
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2))(x)
x = Dropout(0.25)(x)

# Bloque 3
x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2))(x)
x = Dropout(0.25)(x)

# Clasificador
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.4)(x)
output_layer = Dense(10, activation='softmax')(x)

# Modelo
model = Model(inputs=input_layer, outputs=output_layer)
model.summary()

Model: "functional_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_9 (InputLayer)      │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_27 (Conv2D)              │ (None, 32, 32, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_27 (MaxPooling2D) │ (None, 16, 16, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_36 (Dropout)            │ (None, 16, 16, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_28 (Conv2D)              │ (None, 16, 16, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_28 (MaxPooling2D) │ (None, 8, 8, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_37 (Dropout)            │ (None, 8, 8, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_29 (Conv2D)              │ (None, 8, 8, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_29 (MaxPooling2D) │ (None, 4, 4, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_38 (Dropout)            │ (None, 4, 4, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_9 (Flatten)             │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_18 (Dense)                │ (None, 128)            │       262,272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_39 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_19 (Dense)                │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 356,810 (1.36 MB)

 Trainable params: 356,810 (1.36 MB)

 Non-trainable params: 0 (0.00 B)

In [34]:

# Compilación del modelo
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Definición de callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5)
model_checkpoint = ModelCheckpoint('best_model.keras', monitor='val_loss', save_best_only=True)
# Entrenamiento del modelo
history = model.fit(x_train, y_train,
                    validation_split=0.1,
                    epochs=20,
                    batch_size=64,
                    callbacks=[early_stopping, model_checkpoint]
                    )




Epoch 1/20
704/704 ━━━━━━━━━━━━━━━━━━━━ 16s 20ms/step - accuracy: 0.2710 - loss: 1.9616 - val_accuracy: 0.5050 - val_loss: 1.3781
Epoch 2/20
704/704 ━━━━━━━━━━━━━━━━━━━━ 14s 19ms/step - accuracy: 0.4802 - loss: 1.4374 - val_accuracy: 0.5778 - val_loss: 1.1959
Epoch 3/20
704/704 ━━━━━━━━━━━━━━━━━━━━ 14s 20ms/step - accuracy: 0.5420 - loss: 1.2769 - val_accuracy: 0.6224 - val_loss: 1.0983
Epoch 4/20
704/704 ━━━━━━━━━━━━━━━━━━━━ 14s 19ms/step - accuracy: 0.5868 - loss: 1.1582 - val_accuracy: 0.6734 - val_loss: 0.9365
Epoch 5/20
704/704 ━━━━━━━━━━━━━━━━━━━━ 14s 19ms/step - accuracy: 0.6189 - loss: 1.0749 - val_accuracy: 0.6916 - val_loss: 0.8918
Epoch 6/20
704/704 ━━━━━━━━━━━━━━━━━━━━ 14s 19ms/step - accuracy: 0.6406 - loss: 1.0135 - val_accuracy: 0.7162 - val_loss: 0.8238
Epoch 7/20
704/704 ━━━━━━━━━━━━━━━━━━━━ 14s 20ms/step - accuracy: 0.6550 - loss: 0.9762 - val_accuracy: 0.7300 - val_loss: 0.7769
Epoch 8/20
704/704 ━━━━━━━━━━━━━━━━━━━━ 14s 19ms/step - accuracy: 0.6771 - loss: 0.9181 - 

In [ ]:
# Cargar el mejor modelo guardado
model.load_weights('best_model.keras')

# Evaluar el modelo
loss, accuracy = model.evaluate(x_test, y_test, verbose=0)
print(f"Test Accuracy: {accuracy:.2f}")

# Reporte de clasificación
y_pred = model.predict(x_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_test_classes = np.argmax(y_test, axis=1)
print(classification_report(y_test_classes, y_pred_classes))